In [9]:
import numpy as np
from scipy.io import wavfile
import os
from collections import Counter

REVERSE_MORSE_DICT = {
    '.-': 'A', '-...': 'B', '-.-.': 'C', '-..': 'D',
    '.': 'E', '..-.': 'F', '--.': 'G', '....': 'H',
    '..': 'I', '.---': 'J', '-.-': 'K', '.-..': 'L',
    '--': 'M', '-.': 'N', '---': 'O', '.--.': 'P',
    '--.-': 'Q', '.-.': 'R', '...': 'S', '-': 'T',
    '..-': 'U', '...-': 'V', '.--': 'W', '-..-': 'X',
    '-.--': 'Y', '--..': 'Z', '-----': '0', '.----': '1',
    '..---': '2', '...--': '3', '....-': '4', '.....': '5',
    '-....': '6', '--...': '7', '---..': '8', '----.': '9'
}

def smooth_signal(signal, window_size=100):
    return np.convolve(signal, np.ones(window_size)/window_size, mode='same')

def audio_to_morse(filepath, threshold=0.2):
    sr, audio = wavfile.read(filepath)

    if audio.ndim > 1:
        audio = audio[:, 0]

    audio = audio / np.max(np.abs(audio))  # normalize
    envelope = np.abs(audio)
    smoothed = smooth_signal(envelope, window_size=int(sr * 0.01))  # 10 ms window

    is_beep = smoothed > threshold

    durations = []
    current = is_beep[0]
    count = 0

    for bit in is_beep:
        if bit == current:
            count += 1
        else:
            durations.append((current, count))
            current = bit
            count = 1
    durations.append((current, count))

    # Filter out very short blips
    durations = [(state, dur) for state, dur in durations if dur > 10]

    # Estimate base unit duration (mode of beep durations)
    beep_lengths = [d for state, d in durations if state]
    if not beep_lengths:
        print("❌ No valid Morse pulses detected.")
        return ""
    unit = Counter(beep_lengths).most_common(1)[0][0]

    morse = ""
    for state, dur in durations:
        units = round(dur / unit)
        if state:  # beep
            if units <= 2:
                morse += "."
            else:
                morse += "-"
        else:  # silence
            if units >= 7:
                morse += " / "
            elif units >= 3:
                morse += " "
            else:
                pass  # very short silence (between dots/dashes)

    return morse.strip()

def morse_to_text(morse_code):
    words = morse_code.split(" / ")
    text = []
    for word in words:
        letters = word.strip().split()
        decoded = ''.join(REVERSE_MORSE_DICT.get(l, '?') for l in letters)
        text.append(decoded)
    return ' '.join(text)

# 🎯 Main Runner
if __name__ == "__main__":
    filepath = input("📂 Enter path to .wav file (default: output.wav): ").strip()
    if not filepath:
        filepath = "output.wav"

    if not os.path.exists(filepath):
        print("❌ File not found.")
    else:
        print(f"🔍 Analyzing: {filepath}")
        morse = audio_to_morse(filepath)
        print(f"\n📡 Morse Code:\n{morse}")
        text = morse_to_text(morse)
        print(f"\n📝 Decoded Text:\n{text}")



📂 Enter path to .wav file (default: output.wav):  /Users/ishandev/ishan.wav


🔍 Analyzing: /Users/ishandev/ishan.wav

📡 Morse Code:
.. ... .... .- -. / .. ... / .- / -... .- -.. / --. ..- -.--

📝 Decoded Text:
ISHAN IS A BAD GUY
